In [ ]:
from flask import Flask, render_template, request, send_file
from flask import redirect, url_for
import joblib
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from datetime import date
from fpdf import FPDF

app = Flask(__name__)

# Load trained model
model = joblib.load('model.pkl')

# Load dataset for analysis
df = pd.read_csv("water_potability.csv")

# Get current year
year = date.today().year

# Ensure analysis directory exists
analysis_dir = "static/analysis"
os.makedirs(analysis_dir, exist_ok=True)

# Model Accuracies
model_accuracies = {
    "K-Nearest Neighbors (KNN)": 62.36,
    "Decision Tree": 55.65,
    "Random Forest": 64.9,
    "Support Vector Classifier (SVC)": 66.12,
    "XGBoost": 63.78
}

# Identify Best Model
best_model = max(model_accuracies, key=model_accuracies.get)
best_accuracy = model_accuracies[best_model]


@app.route("/")
def index():
    return render_template("index.html")

@app.route("/predict", methods=["GET", "POST"])
def predict():
    if request.method == "POST":
        try:
            # Extract form data
            data = [
                float(request.form["ph"]),
                float(request.form["hardness"]),
                float(request.form["solids"]),
                float(request.form["chloramines"]),
                float(request.form["sulfate"]),
                float(request.form["conductivity"]),
                float(request.form["organic_carbon"]),
                float(request.form["trihalomethanes"]),
                float(request.form["turbidity"])
            ]
            
            # Make prediction
            prediction = model.predict([data])[0]
            prediction_text = (
                "The water is potable and meets safety standards."
                if prediction == 0 else
                "The water is non-potable and may be unsafe to drink."
            )
            
            return render_template("predict.html", current_year=year, prediction_text=prediction_text)
        except Exception as e:
            return render_template("error.html", error=str(e))
    return render_template("predict.html", current_year=year)

@app.route('/pairplot')
def pairplot():
    pairplot_path = "static/pairplot.png"
    if not os.path.exists(pairplot_path):
        return "Pairplot image not found. Please generate it first."
    return render_template("pairplot.html")

@app.route("/data_analysis")
def data_analysis():
    os.makedirs(analysis_dir, exist_ok=True)
    for file in os.listdir(analysis_dir):
        if file.endswith(".png"):
            os.remove(os.path.join(analysis_dir, file))
    
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        try:
            fig, axes = plt.subplots(1, 2, figsize=(6, 3))
            sns.histplot(df[col], kde=True, color="blue", ax=axes[0])
            axes[0].set_title(f"Distribution of {col}")
            sns.boxplot(x=df[col], color="green", ax=axes[1])
            axes[1].set_title(f"Box Plot of {col}")
            plt.tight_layout()
            img_path = f"{analysis_dir}/{col}_analysis.png"
            plt.savefig(img_path, dpi=50)
            plt.close()
        except Exception as e:
            print(f"Error processing {col}: {e}")
    
    images = [f"analysis/{img}" for img in os.listdir(analysis_dir) if img.endswith(".png")]
    return render_template("data_analysis.html", images=images, current_year=year)

@app.route("/report")
def report():
    return render_template("report.html")

@app.route("/generate_report")
def generate_report():
    pdf = FPDF(orientation='L', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_font("Arial", 'B', 18)
    pdf.cell(280, 10, "Water Quality Analysis Report", ln=True, align='C')
    pdf.set_font("Arial", 'I', 12)
    pdf.cell(280, 7, "A summary report on water quality parameters", ln=True, align='C')
    pdf.ln(10)
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(280, 7, "Summary Statistics", ln=True, align='C')
    pdf.ln(6)
    
    pdf.set_font("Arial", '', 9)
    summary = df.describe().round(2)
    col_width = 280 / len(summary.columns)
    pdf.set_fill_color(200, 220, 255)
    pdf.set_font("Arial", 'B', 8)
    for col in summary.columns:
        pdf.cell(col_width, 7, col, border=1, align='C', fill=True)
    pdf.ln()
    pdf.set_font("Arial", '', 8)
    for index, row in summary.iterrows():
        pdf.cell(col_width, 7, index, border=1, align='C', fill=True)
        for col in summary.columns:
            pdf.cell(col_width, 7, str(row[col]), border=1, align='C')
        pdf.ln()
    
    pdf.ln(10)
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(280, 7, "Recommendations", ln=True, align='C')
    pdf.ln(8)
    pdf.set_font("Arial", '', 10)
    recommendations = [
        "1. Maintain pH between 6.5 and 8.5.",
        "2. High hardness levels may require a water softener.",
        "3. Use activated carbon filters for high chloramine levels.",
        "4. Regularly test water for contaminants.",
        "5. Use reverse osmosis for high sulfate levels.",
    ]
    for rec in recommendations:
        pdf.multi_cell(0, 7, rec, align='L')
        pdf.ln(1)
    pdf.ln(10)
    
    report_path = "static/water_quality_report.pdf"
    pdf.output(report_path)
    return send_file(report_path, as_attachment=True)

@app.route('/heatmap')
def heatmap():
    heatmap_path = "static/heatmap.png"

    # Create heatmap
    plt.figure(figsize=(12, 8), dpi=150)
    sns.heatmap(df.corr(), annot=True, cmap='viridis', vmin=-1, vmax=1, linewidths=0.5)
    
    # Save the heatmap
    plt.savefig(heatmap_path, bbox_inches='tight')
    plt.close()

    return render_template("heatmap.html", heatmap_image=heatmap_path)

@app.route("/clustermap")
def clustermap():
    plot_path = "static/clustermap.png"

    # Generate the clustermap if it doesn't exist
    if not os.path.exists(plot_path):
        plt.figure(figsize=(10, 8))
        sns.clustermap(df.corr(), cmap="magma", annot=False, linewidths=0.5)
        plt.savefig(plot_path, dpi=300, bbox_inches="tight")
        plt.close()

    return render_template("clustermap.html", plot_path=plot_path)

@app.route('/model_description')
def model_description():
    return render_template("model_description.html", model_accuracies=model_accuracies, best_model=best_model, best_accuracy=best_accuracy)

@app.route('/testing-centres')
def testing_centres():
    return render_template('testing-centres.html')

@app.route('/filter-recommender', methods=['GET', 'POST'])
def filter_recommender():
    recommendation = None

    if request.method == 'POST':
        ph = float(request.form['ph'])
        turbidity = float(request.form['turbidity'])
        tds = float(request.form['tds'])
        hardness = float(request.form['hardness'])
        chlorine = float(request.form['chlorine'])
        sulfate = float(request.form['sulfate'])
        conductivity = float(request.form['conductivity'])

        # Recommendation logic based on water quality
        if ph < 6.5 or ph > 8.5:
            recommendation = "Reverse Osmosis (RO) Filter - Best for pH imbalance."
        elif turbidity > 5:
            recommendation = "Sediment & Carbon Filters - Effective for high turbidity."
        elif tds > 500:
            recommendation = "RO + UV Filter - Best for high TDS levels."
        elif hardness > 200:
            recommendation = "Water Softener - Reduces hardness effectively."
        elif chlorine > 4:
            recommendation = "Activated Carbon Filter - Removes excess chlorine."
        elif sulfate > 250:
            recommendation = "Ion Exchange Filter - Reduces sulfate levels."
        elif conductivity > 1000:
            recommendation = "Advanced RO System - Best for high conductivity."
        else:
            recommendation = "Standard Carbon Filter - Suitable for normal water."

    return render_template('filter_recommender.html', recommendation=recommendation)

@app.route('/dataset')
def dataset():
    table_html = df.to_html(classes="table table-striped table-bordered", index=False)
    return render_template('dataset.html', table=table_html)


if __name__ == "__main__":
    app.run( )


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
